In [1]:
%run ../load_files.ipynb

In [2]:
%run ../data_all_matrixes.ipynb

(98, 39)


In [3]:
%run ../y_multilevel.ipynb

,attention,memory
ID,,
FIS_001,0,0
FIS_002,0,0
FIS_004,0,0
FIS_005,0,0
FIS_007,1,0


In [4]:
y = y_values.memory

In [5]:
# Training and test subsamples

from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(df, y, test_size=0.2, stratify=y)

In [6]:
print(X_train.shape)
print(X_test.shape)
print(y_train.shape)
print(y_test.shape)

(78, 39)
(20, 39)
(78,)
(20,)


In [7]:
# Keras model

from keras.models import Sequential
from keras.layers import Dense, Activation, InputLayer
from keras.optimizers import SGD
from keras.wrappers.scikit_learn import KerasClassifier
from scipy import random
from sklearn.model_selection import RandomizedSearchCV, StratifiedKFold
from keras.wrappers.scikit_learn import KerasClassifier

n_classes = 2


def create_model(n_hidden, learning_rate):
    model = Sequential()
    model.add(InputLayer(input_shape=(2850, )))
    model.add(Dense(units=n_hidden))
    model.add(Activation("sigmoid"))
    model.add(Dense(n_classes))
    model.add(Activation("softmax"))
    sgd = SGD(lr=learning_rate)
    model.compile(loss='categorical_crossentropy', optimizer=sgd, metrics=["accuracy"])
    return model

params = {
    'n_hidden': random.randint(5, 200, 100),
    'learning_rate': random.uniform(0.001, 0.2, 100),
    'epochs': random.randint(10, 50, 100),
}
fn_model = KerasClassifier(build_fn=create_model)

from sklearn.pipeline import Pipeline
from sklearn.svm import LinearSVC
from sklearn.decomposition import PCA
from sklearn.linear_model import LogisticRegression
from sklearn.feature_selection import SelectFromModel

from neudataload.transformer import CombineMatrixTransformer, SpreadOutMatrixTransformer, FeatureMatrixTransformer
from neudataload.utils import all_combinations

# Pipeline configuration
pipe = Pipeline([
    ('combining', CombineMatrixTransformer(column_name='combined')),
    ('spread_out', SpreadOutMatrixTransformer(columns=['combined'])),
    ('filter', FeatureMatrixTransformer(matrix_columns=['combined'])),

    # ('reduce_dim', PCA(n_components=43)),
    ('classify', fn_model)
])


# Parametrized values
N_FEATURES_OPTIONS = range(1, 500, 10)
MATRIXES = list(all_combinations(column_matrixes))
EXTRA_1D_COLUMS = [None]                      


param_grid = { 
        'combining__columns': MATRIXES,
        # 'filter__columns': EXTRA_1D_COLUMS,
        
        'classify__n_hidden': random.randint(20, 200, 1000),
        'classify__learning_rate': random.uniform(0.001, 0.2, 1000),
        'classify__epochs': random.randint(10, 50, 1000),
    }

Using TensorFlow backend.


In [8]:
from sklearn.model_selection import RandomizedSearchCV
from sklearn.model_selection import StratifiedKFold

# Search the best parameter combination with Cross Validation 
cv = StratifiedKFold(n_splits=3, shuffle=True)

grid = RandomizedSearchCV(pipe, cv=cv, n_jobs=-1, param_distributions=param_grid, 
                          verbose=3,
                          scoring='accuracy',
                          n_iter = 150)


In [9]:
grid.fit(X_train, y_train)

Fitting 3 folds for each of 150 candidates, totalling 450 fits
[CV] combining__columns=('DTI_FA', 'DTI_MD', 'DTI_RX', 'LS'), classify__n_hidden=27, classify__learning_rate=0.03818942835968997, classify__epochs=20 
[CV] combining__columns=('DTI_FA', 'DTI_MD', 'DTI_RX', 'LS'), classify__n_hidden=27, classify__learning_rate=0.03818942835968997, classify__epochs=20 
[CV] combining__columns=('DTI_FA', 'DTI_MD', 'DTI_RX', 'LS'), classify__n_hidden=27, classify__learning_rate=0.03818942835968997, classify__epochs=20 
[CV] combining__columns=('DTI_L1', 'DTI_MD', 'LS'), classify__n_hidden=87, classify__learning_rate=0.009001749149093703, classify__epochs=12 
[CV] combining__columns=('DTI_L1', 'DTI_MD', 'LS'), classify__n_hidden=87, classify__learning_rate=0.009001749149093703, classify__epochs=12 
[CV] combining__columns=('DTI_L1', 'DTI_MD', 'LS'), classify__n_hidden=87, classify__learning_rate=0.009001749149093703, classify__epochs=12 
[CV] combining__columns=('DTI_MD', 'DTI_RX', 'RAW', 'LS'),

32/53 [=================>............] - ETA: 0s - loss: 0.6156 - acc: 0.7188Epoch 7/12
Epoch 13/20
52/52 [==============================] - 0s 195us/step - loss: 0.5517 - acc: 0.7115
Epoch 19/20
52/52 [==============================] - 0s 314us/step - loss: 0.7100 - acc: 0.6154
Epoch 14/20
52/52 [==============================] - 0s 288us/step - loss: 0.5604 - acc: 0.6923
Epoch 20/20
53/53 [==============================] - 0s 327us/step - loss: 0.6041 - acc: 0.6415
Epoch 8/12
Epoch 15/20
Epoch 4/12
53/53 [==============================] - 0s 160us/step - loss: 0.6139 - acc: 0.6226
Epoch 5/12
52/52 [==============================] - 0s 458us/step - loss: 0.7235 - acc: 0.5769
Epoch 4/12
Epoch 16/20
51/51 [==============================] - 0s 426us/step - loss: 0.4540 - acc: 0.8431
Epoch 1/13
Epoch 6/12
Epoch 10/12
Epoch 5/12
53/53 [==============================] - 0s 318us/step - loss: 0.5992 - acc: 0.6415
Epoch 17/20
53/53 [==============================] - 0s 418us/step - loss: 0.49

[CV]  combining__columns=('DTI_L1', 'DTI_MD', 'LS'), classify__n_hidden=87, classify__learning_rate=0.009001749149093703, classify__epochs=12, score=0.5555555555555556, total=   1.6s
[CV]  combining__columns=('DTI_FA', 'DTI_MD', 'DTI_RX', 'LS'), classify__n_hidden=27, classify__learning_rate=0.03818942835968997, classify__epochs=20, score=0.68, total=   1.6s
[CV]  combining__columns=('DTI_L1', 'DTI_MD', 'LS'), classify__n_hidden=87, classify__learning_rate=0.009001749149093703, classify__epochs=12, score=0.64, total=   1.5s
[CV] combining__columns=('DTI_FA', 'DTI_L1'), classify__n_hidden=87, classify__learning_rate=0.12632281843078064, classify__epochs=33 
[CV] combining__columns=('DTI_FA', 'DTI_L1'), classify__n_hidden=87, classify__learning_rate=0.12632281843078064, classify__epochs=33 
[CV]  combining__columns=('DTI_L1', 'DTI_MD', 'LS'), classify__n_hidden=87, classify__learning_rate=0.009001749149093703, classify__epochs=12, score=0.46153846153846156, total=   1.7s
[CV] combining__

51/51 [==============================] - 0s 251us/step - loss: 0.6126 - acc: 0.6471
Epoch 19/33
53/53 [==============================] - 0s 180us/step - loss: 0.6514 - acc: 0.6415
Epoch 11/33
Epoch 6/38
51/51 [==============================] - 0s 201us/step - loss: 0.6289 - acc: 0.5490
Epoch 11/33
Epoch 20/33
Epoch 1/34
Epoch 12/33
52/52 [==============================] - 0s 253us/step - loss: 0.7028 - acc: 0.6538
Epoch 13/33
Epoch 12/33
Epoch 21/33
52/52 [==============================] - 0s 402us/step - loss: 0.7558 - acc: 0.4423
Epoch 8/38
Epoch 22/33
Epoch 14/33
Epoch 13/33
52/52 [==============================] - 0s 301us/step - loss: 0.6539 - acc: 0.5962
Epoch 3/38
51/51 [==============================] - 0s 360us/step - loss: 0.6296 - acc: 0.6863
Epoch 15/33
Epoch 23/33
51/51 [==============================] - 0s 415us/step - loss: 0.8160 - acc: 0.4706
Epoch 9/38
51/51 [==============================] - 0s 194us/step - loss: 0.6107 - acc: 0.6471
Epoch 4/38
51/51 [===============

53/53 [==============================] - 0s 164us/step - loss: 0.7308 - acc: 0.5283
Epoch 6/38
52/52 [==============================] - 0s 506us/step - loss: 0.5978 - acc: 0.6538
Epoch 3/34
Epoch 25/33
51/51 [==============================] - 0s 139us/step - loss: 0.6277 - acc: 0.6863
Epoch 22/38
Epoch 15/38
Epoch 26/33
52/52 [==============================] - 0s 182us/step - loss: 0.5758 - acc: 0.7308
Epoch 27/33
53/53 [==============================] - 0s 396us/step - loss: 0.7240 - acc: 0.6226
Epoch 8/38
52/52 [==============================] - 0s 201us/step - loss: 0.5506 - acc: 0.6923
Epoch 16/38
Epoch 28/33
Epoch 23/38
Epoch 25/33
51/51 [==============================] - 0s 242us/step - loss: 0.8028 - acc: 0.5490
Epoch 5/34
51/51 [==============================] - 0s 452us/step - loss: 0.6610 - acc: 0.7059
Epoch 24/38
53/53 [==============================] - 0s 395us/step - loss: 0.6757 - acc: 0.6415
Epoch 17/38
Epoch 9/38
53/53 [==============================] - 0s 354us/step - 

...

Epoch 32/37
51/51 [==============================] - 0s 225us/step - loss: 0.6832 - acc: 0.6471
Epoch 33/37
Epoch 9/37
52/52 [==============================] - 0s 459us/step - loss: 0.8449 - acc: 0.4423
Epoch 34/37
Epoch 10/37
51/51 [==============================] - 0s 403us/step - loss: 0.7668 - acc: 0.4510
[CV]  combining__columns=('DTI_FA', 'RAW', 'LS'), classify__n_hidden=61, classify__learning_rate=0.1706834898135872, classify__epochs=37, score=0.64, total=   4.2s
Epoch 35/37
52/52 [==============================] - 0s 379us/step - loss: 0.8687 - acc: 0.4808
Epoch 11/37
52/52 [==============================] - 0s 297us/step - loss: 0.7762 - acc: 0.4808
Epoch 36/37
Epoch 12/37
51/51 [==============================] - 0s 493us/step - loss: 0.6483 - acc: 0.6471
Epoch 37/37
52/52 [==============================] - 0s 484us/step - loss: 0.6811 - acc: 0.6346
Epoch 13/37
52/52 [==============================] - 0s 432us/step - loss: 0.7634 - acc: 0.4808
Epoch 14/37
52/52 [==============

[CV]  combining__columns=('DTI_MD', 'DTI_RX', 'RAW'), classify__n_hidden=86, classify__learning_rate=0.162048371281018, classify__epochs=37, score=0.64, total=   2.4s


[Parallel(n_jobs=-1)]: Done 450 out of 450 | elapsed:  3.9min finished


Epoch 1/29
78/78 [==============================] - 0s 960us/step - loss: 0.8355 - acc: 0.4744
Epoch 2/29
78/78 [==============================] - 0s 124us/step - loss: 0.6880 - acc: 0.6410
Epoch 3/29
78/78 [==============================] - 0s 122us/step - loss: 0.6710 - acc: 0.5897
Epoch 4/29
78/78 [==============================] - 0s 103us/step - loss: 0.6479 - acc: 0.6410
Epoch 5/29
78/78 [==============================] - 0s 105us/step - loss: 0.6388 - acc: 0.6410
Epoch 6/29
78/78 [==============================] - 0s 116us/step - loss: 0.6451 - acc: 0.6154
Epoch 7/29
78/78 [==============================] - 0s 102us/step - loss: 0.6536 - acc: 0.6667
Epoch 8/29
78/78 [==============================] - 0s 118us/step - loss: 0.7099 - acc: 0.6154
Epoch 9/29
78/78 [==============================] - 0s 110us/step - loss: 0.6785 - acc: 0.6410
Epoch 10/29
78/78 [==============================] - 0s 95us/step - loss: 0.6210 - acc: 0.6795
Epoch 11/29
78/78 [==============================]

RandomizedSearchCV(cv=StratifiedKFold(n_splits=3, random_state=None, shuffle=True),
          error_score='raise',
          estimator=Pipeline(memory=None,
     steps=[('combining', CombineMatrixTransformer(column_name='combined', columns=None,
             op=<function mean at 0x7ff93c106840>)), ('spread_out', SpreadOutMatrixTransformer(columns=['combined'], symmetric=True)), ('filter', FeatureMatrixTransformer(columns=None, matrix_columns=['combined'])), ('classify', <keras.wrappers.scikit_learn.KerasClassifier object at 0x7ff8ffa7f7b8>)]),
          fit_params=None, iid=True, n_iter=150, n_jobs=-1,
          param_distributions={'combining__columns': [('DTI_FA',), ('DTI_L1',), ('DTI_MD',), ('DTI_RX',), ('RAW',), ('LS',), ('DTI_FA', 'DTI_L1'), ('DTI_FA', 'DTI_MD'), ('DTI_FA', 'DTI_RX'), ('DTI_FA', 'RAW'), ('DTI_FA', 'LS'), ('DTI_L1', 'DTI_MD'), ('DTI_L1', 'DTI_RX'), ('DTI_L1', 'RAW'), ('DTI_L1', 'LS'), (... array([0.097  , 0.15184, ..., 0.06244, 0.09943]), 'classify__epochs': array(

In [10]:
%run ../grid_results.ipynb

combining__columns = ('DTI_FA', 'DTI_MD')
classify__n_hidden = 72
classify__learning_rate = 0.11955873107296836
classify__epochs = 29
Epoch 1/29
78/78 [==============================] - 0s 1ms/step - loss: 0.6666 - acc: 0.5513
Epoch 2/29
78/78 [==============================] - 0s 103us/step - loss: 0.6812 - acc: 0.6410
Epoch 3/29
78/78 [==============================] - 0s 122us/step - loss: 0.6628 - acc: 0.6410
Epoch 4/29
78/78 [==============================] - 0s 106us/step - loss: 0.6705 - acc: 0.6410
Epoch 5/29
78/78 [==============================] - 0s 153us/step - loss: 0.6495 - acc: 0.6410
Epoch 6/29
78/78 [==============================] - 0s 100us/step - loss: 0.6965 - acc: 0.5641
Epoch 7/29
78/78 [==============================] - 0s 146us/step - loss: 0.6357 - acc: 0.6410
Epoch 8/29
78/78 [==============================] - 0s 97us/step - loss: 0.8184 - acc: 0.4359
Epoch 9/29
78/78 [==============================] - 0s 150us/step - loss: 0.6288 - acc: 0.6410
Epoch 10/29
78

/home/efrain/.virtualenvs/NeuDataLoad/lib/python3.6/site-packages/sklearn/metrics/classification.py:1135: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)
